In [1]:
from imports import *
from PreProcessor import *

_PreProcessor = PreProcessor()


c:\Users\basse\anaconda3\envs\cuda_env\lib\site-packages\tree_sitter\__init__.py:36: FutureWarning: Language.build_library is deprecated. Use the new bindings instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)
c:\Users\basse\anaconda3\envs\cuda_env\lib\site-packages\tree_sitter\__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


In [2]:
function_names_train,function_segments_train = _PreProcessor.setupDeepLearning("./dataset/teiid_dataset/train_CC",'CC','train')
descriptions_train, summaries_train = _PreProcessor.setupDeepLearning("./dataset/teiid_dataset/train_UC",'UC','train')
function_names_test,function_segments_test = _PreProcessor.setupDeepLearning("./dataset/teiid_dataset/test_CC",'CC','test')
descriptions_test, summaries_test = _PreProcessor.setupDeepLearning("./dataset/teiid_dataset/test_UC",'UC','test')


In [3]:
_PreProcessor.setUpUnknown(function_names_train,function_segments_train,'CC')
_PreProcessor.setUpUnknown(descriptions_train,summaries_train,'UC')

_PreProcessor.setUpUnknown(function_names_test,function_segments_test,'CC')
_PreProcessor.setUpUnknown(descriptions_test,summaries_test,'UC')

In [5]:
with open('./pickles/DeepLearning/function_names_train.pkl', 'wb') as f:
       pickle.dump(function_names_train, f)
with open('./pickles/DeepLearning/function_segments_train.pkl', 'wb') as f:
        pickle.dump(function_segments_train, f)
with open('./pickles/DeepLearning/descriptions_train.pkl', 'wb') as f:
        pickle.dump(descriptions_train, f)
with open('./pickles/DeepLearning/summaries_train.pkl', 'wb') as f:
        pickle.dump(summaries_train, f)

with open('./pickles/DeepLearning/Vocab.pkl', 'wb') as f:
        pickle.dump(_PreProcessor.Vocab, f)

with open('./pickles/DeepLearning/function_names_test.pkl', 'wb') as f:
       pickle.dump(function_names_test, f)
with open('./pickles/DeepLearning/function_segments_test.pkl', 'wb') as f:
        pickle.dump(function_segments_test, f)
with open('./pickles/DeepLearning/descriptions_test.pkl', 'wb') as f:
        pickle.dump(descriptions_test, f)
with open('./pickles/DeepLearning/summaries_test.pkl', 'wb') as f:
        pickle.dump(summaries_test, f)

In [2]:
function_names_train = np.load('./pickles/DeepLearning/function_names_train.pkl',allow_pickle=True)
function_segments_train = np.load('./pickles/DeepLearning/function_segments_train.pkl',allow_pickle=True)
descriptions_train = np.load('./pickles/DeepLearning/descriptions_train.pkl',allow_pickle=True)
summaries_train = np.load('./pickles/DeepLearning/summaries_train.pkl',allow_pickle=True)
vocab = np.load('./pickles/DeepLearning/Vocab.pkl',allow_pickle=True)
function_names_test = np.load('./pickles/DeepLearning/function_names_test.pkl',allow_pickle=True)
function_segments_test = np.load('./pickles/DeepLearning/function_segments_test.pkl',allow_pickle=True)
descriptions_test = np.load('./pickles/DeepLearning/descriptions_test.pkl',allow_pickle=True)
summaries_test = np.load('./pickles/DeepLearning/summaries_test.pkl',allow_pickle=True)


In [3]:
CC_word2vec = _PreProcessor.word2VecProcessor(function_names_train, function_segments_train, 'CC')
UC_word2vec = _PreProcessor.word2VecProcessor(summaries_train, descriptions_train, 'UC')
# print(CC_word2vec)
word2vec_model = Word2Vec(sentences = CC_word2vec + UC_word2vec, vector_size=200, window=5, min_count=1, workers=4, epochs=10)
word2vec_model.save("./Dataset/teiid_dataset/UC_CC_WORD2VEC")

#word2vec_model.build_vocab(UC_word2vec, update=True)
#word2vec_model.train(UC_word2vec, total_examples=word2vec_model.corpus_count, epochs=word2vec_model.epochs)

# word2vec_model.save("./Dataset/teiid_dataset/UC_CC_WORD2VEC")
word2vec_model = Word2Vec.load("./Dataset/teiid_dataset/UC_CC_WORD2VEC")

word2vec_vocab = word2vec_model.wv.index_to_key

# Initialize an empty embedding matrix
embedding_matrix = np.zeros((len(word2vec_vocab) + 1, word2vec_model.vector_size))

# Fill the embedding matrix with the embeddings of each word
for i, word in enumerate(word2vec_vocab):
    embedding_vector = word2vec_model.wv[word]
    if embedding_vector is not None:
        embedding_matrix[i + 1] = embedding_vector



In [8]:
print(len(embedding_matrix))

6245


### Converting dataset to indices

In [9]:
_PreProcessor.vocabToIndex(word2vec_vocab)
_PreProcessor.dataSetToIndex(function_names_train,function_segments_train)
_PreProcessor.dataSetToIndex(descriptions_train,summaries_train)

_PreProcessor.dataSetToIndex(function_names_test,function_segments_test)
_PreProcessor.dataSetToIndex(descriptions_test,summaries_test)

In [10]:
x_train, labels = _PreProcessor.setUpLabels(function_names_train,function_segments_train,descriptions_train,summaries_train)

In [11]:
with open('./pickles/DeepLearning/x_train.pkl', 'wb') as f:
       pickle.dump(x_train, f)
with open('./pickles/DeepLearning/labels.pkl', 'wb') as f:
        pickle.dump(labels, f)

In [4]:
x_train = np.load('./pickles/DeepLearning/x_train.pkl',allow_pickle=True)
labels = np.load('./pickles/DeepLearning/labels.pkl',allow_pickle=True)


In [5]:
class TracibilityLinkDataset(Dataset):

    def __init__(self, x_train, y_train):
        
        self.tensor_x_train = x_train
        self.tensor_y_train = torch.tensor(y_train)


    # len and get item are very important --> used by dataloader
    def __len__(self):
        return len(self.tensor_y_train)

    def __getitem__(self, idx):
        return self.tensor_x_train[idx], self.tensor_y_train[idx]


In [6]:
dataset_train = TracibilityLinkDataset(x_train, labels)


In [7]:
# torch.save(dataset_train,'./dataset/teiid_dataset/DeepLearningDataset.pt')


In [8]:
# dataset_train = torch.load('./dataset/teiid_dataset/DeepLearningDataset.pt')



In [9]:
import torch.utils.checkpoint as checkpoint 
class DLModel(nn.Module):

  def __init__(self,embedding_matrix : np.array, embedding_dim : tuple, classes: int = 2):
      super(DLModel, self).__init__()

      # first embedding layer: input size --> vocab size, output size --> feature vector of vocan size(one hot encoding), weigths --> one hot encoding
      # matrix (vocab * vocab)
      self.embedding = nn.Embedding(num_embeddings = embedding_dim[0], embedding_dim = embedding_dim[1], _weight = torch.tensor(embedding_matrix))
      
      # self-attention layer --> num_heads = 1, query, key, value will all be the input
      self.attention_layer_UC = nn.MultiheadAttention(embed_dim=embedding_dim[1], num_heads=1, batch_first = True)
      self.attention_layer_CC = nn.MultiheadAttention(embed_dim=embedding_dim[1], num_heads=1, batch_first = True)

      self.conv_layer_2d =torch.nn.Conv2d(1, 1, kernel_size = (5,5), stride = (2,2))
      
      self.pool_layer_2d = torch.nn.AvgPool2d(kernel_size = (5,5), stride = (2, 2))
      
      self.cosine_similarity = torch.nn.CosineSimilarity(dim=1) # code = 1*2000*200 , req = code = 1*1000*200 


  def forward(self, function_names, function_segments, descriptions, summaries ):

    function_names_embeddings  = self.embedding(function_names)
    function_segments_embeddings = self.embedding(function_segments)
    descriptions_embedding  = self.embedding(descriptions)
    summaries_embedding = self.embedding(summaries)

    function_names_attention = self.attention_layer_CC(function_names_embeddings.float(),function_names_embeddings.float(),function_names_embeddings.float())

    function_segments_attention = self.attention_layer_CC(function_segments_embeddings.float(),function_segments_embeddings.float(),function_segments_embeddings.float())
   
    descriptions_attention = self.attention_layer_UC(descriptions_embedding.float(),descriptions_embedding.float(),descriptions_embedding.float())

    summaries_attention = self.attention_layer_UC(summaries_embedding.float(),summaries_embedding.float(),summaries_embedding.float())

    concatenated_name_seg = torch.cat((function_names_attention[0], function_segments_attention[0]), dim = 1)

    summaries_descrip_seg = torch.cat((descriptions_attention[0], summaries_attention[0]), dim = 1)

    names_seg_conv = self.conv_layer_2d(concatenated_name_seg.unsqueeze(1))
    # print("names_seg_conv.shape = " , names_seg_conv.shape)

    # summaries_descrip_conv = self.conv_layer_2d(summaries_descrip_seg.unsqueeze(1))
    # print("summaries_descrip_seg.shape = ",summaries_descrip_seg.shape)

    name_seg = self.pool_layer_2d (names_seg_conv)
    # print("name_seg.shape = " , name_seg.shape)

    summaries_descrip_flatten = summaries_descrip_seg.view(summaries_descrip_seg.shape[0],-1)
    name_seg_flatten = name_seg.view(name_seg.shape[0],-1) 

    # print("name_seg_flatten.shape = " , name_seg_flatten.shape)
    # print("summaries_descrip_flatten.shape = " , summaries_descrip_flatten.shape)

    num_repeats = (name_seg_flatten.shape[1] + summaries_descrip_flatten.shape[1] - 1) // summaries_descrip_flatten.shape[1] 

    #inreasing th size of the req document to be the same as the code documet to allow for cosine simiraity 
    summaries_descrip_flatten_tiled = summaries_descrip_flatten.repeat((1,num_repeats))
    summaries_descrip_flatten_tiled_segmented = summaries_descrip_flatten_tiled[:,:name_seg_flatten.shape[1]]
    # print("summaries_descrip_flatten_tiled.shape",summaries_descrip_flatten_tiled_segmented.shape)

    consine_sim_code_req = self.cosine_similarity(name_seg_flatten,summaries_descrip_flatten_tiled_segmented)

    # summary_desc = self.pool_layer_2d (summaries_descrip_seg)
    # print("summary_desc.shape = ",summary_desc.shape)

    # print(torch.cuda.memory_reserved()/1024**3)
    
    # print("consine_sim_code_req.shape", consine_sim_code_req.shape)
    return consine_sim_code_req

In [10]:
from torch.nn.utils.rnn import pad_sequence

def customCollate(batch: list):
    # batch --> tuple (x,y)
    # x --> list of 4 lists
    x_batch, y_batch = zip(*batch)

    function_names, function_segments, descriptions , summaries = zip(*x_batch)

    function_names_padded = pad_sequence(function_names, batch_first=True, padding_value=0)
    function_segments_padded = pad_sequence(function_segments, batch_first=True, padding_value = 0)
    descriptions_padded = pad_sequence(descriptions, batch_first=True, padding_value=0)
    summaries_padded = pad_sequence(summaries, batch_first=True, padding_value = 0)

    return  function_names_padded, function_segments_padded, descriptions_padded, summaries_padded, y_batch

def train(model, train_dataset, batch_size=40, epochs=1, learning_rate=0.01):

    # (1) create the dataloader of the training set IMPORTANT (make the shuffle=True)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, collate_fn=customCollate)

    # (2) make the criterion cross entropy loss
    criterion = torch.nn.BCELoss() 
    optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu") 
    # device= "cpu"
    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    # (3) create the optimizer (Adam)

    for epoch_num in range(epochs):
        total_acc_train = 0
        total_loss_train = 0
        n_samples = 0
        batch_nu= 0
        for function_names_padded, function_segments_padded, descriptions_padded, summaries_padded, y_batch in train_dataloader:
            # torch.cuda.empty_cache()     
            function_names_padded_tensor = torch.tensor(function_names_padded, device=device)
            function_segments_padded_tensor = torch.tensor(function_segments_padded, device=device)
            descriptions_padded_tensor = torch.tensor(descriptions_padded, device=device)
            fsummaries_padded_tensor = torch.tensor(summaries_padded, device=device)
            y_batch_tensor = torch.tensor(y_batch,dtype = torch.float32,device=device)

            output = model.forward(function_names_padded_tensor, function_segments_padded_tensor, descriptions_padded_tensor, fsummaries_padded_tensor)
            batch_nu += 1

            # print(batch_nu)
            
            # torch.cuda.empty_cache()

            # print(torch.cuda.memory_reserved()/1024**3)
              

            # (7) loss calculation (you need to think in this part how to calculate the loss correctly)
            output_0_1 = torch.tensor((output > 0.5), dtype=torch.float32)
            # print(type(output_0_1[0]))
            # print(output_0_1)
            # print(y_batch)
            # print(type(y_batch[0]))

            batch_loss = criterion(output_0_1, y_batch_tensor)
            print(batch_loss)
            #(8) append the batch loss to the total_loss_train

            total_loss_train += batch_loss
            
    #       (9) calculate the batch accuracy (just add the number of correct predictions)
            # print(output_0_1)
            # print(y_batch)
            n_true = sum((output_0_1 == y_batch))
            n_samples += y_batch.size(0)
            total_acc_train += n_true



    #       (11) do the backward pass
            batch_loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    #       (12) update the weights with your optimizer
            optimizer.step()
            # (10) zero your gradients
            optimizer.zero_grad()
            

            print("after forward = " , torch.cuda.memory_reserved()/1024**3)

    #   epoch loss
        epoch_loss = total_loss_train / y_batch.size(0)

    #   (13) calculate the accuracy
        epoch_acc =100*total_acc_train/n_samples

        print(
           f'Epochs: {epoch_num + 1} | Train Loss: {epoch_loss} \
           | Train Accuracy: {epoch_acc}\n')
        break

  ##############################################################################################################

In [11]:
model = DLModel(embedding_matrix, embedding_matrix.shape)
train(model, dataset_train)


C:\Users\basse\AppData\Local\Temp\ipykernel_25248\2790269522.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  function_names_padded_tensor = torch.tensor(function_names_padded, device=device)
C:\Users\basse\AppData\Local\Temp\ipykernel_25248\2790269522.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  function_segments_padded_tensor = torch.tensor(function_segments_padded, device=device)
C:\Users\basse\AppData\Local\Temp\ipykernel_25248\2790269522.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  descriptions_padded_tensor = tor

tensor(0., device='cuda:0')


TypeError: 'bool' object is not iterable

In [ ]:
import gc
gc.collect()

In [ ]:
t1=torch.tensor([1,1,0,1])
t2=torch.tensor([1,0,0,1])
print(t1==t2)

tensor([ True, False,  True,  True])
